In [1]:
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 40.1 MB/s eta 0:00:00


In [2]:
import math
import urllib
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Capstone/datasets/user_data_source_dropped.csv", sep=';')

In [4]:
df.head()

,Unnamed: 0.1,Unnamed: 0,is_scammer,trx_date,report_date,registereddate,birthday,is_verified,aqc_freq_prepaid_mobile,aqc_mean_prepaid_mobile_amount,...,gender_None,job_position_KARYAWAN,job_position_LAINNYA,job_position_PEGAWAI_NS,job_position_PELAJAR,job_position_RUMAH_TANGGA,job_position_SPESIALIS,job_position_TIDAK_KERJA,job_position_WIRASWASTA,uid
0,0,0,1,0.376934,0.005629,0.673512,0.909517,1,0.019455,0.456958,...,0,0,0,0,1,0,0,0,0,a265c05d-6ed6-4991-971e-6ffab1074379
1,1,1,0,0.552039,0.632270,0.784524,0.948204,1,0.000000,0.000000,...,0,0,0,0,1,0,0,0,0,4d6ebe72-f050-49fd-82fe-a8764bb7a1a2
2,2,2,0,0.772855,0.696998,0.783631,0.766422,1,0.077821,0.324991,...,0,0,0,0,0,0,0,0,1,08744e6e-fec5-4168-bf38-68438dec2f88
3,3,3,0,0.535865,0.550657,0.565476,0.974102,1,0.042802,0.200904,...,0,0,0,0,1,0,0,0,0,f2a873e9-1e28-4db6-9652-cede968ab5d5
4,4,4,0,0.786920,0.715760,0.999702,0.688586,1,0.000000,0.000000,...,0,0,0,0,0,0,0,0,1,c78d7429-7173-4c60-b911-dd8b10246baa


In [5]:
df1 = df.drop(columns=['Unnamed: 0', 'uid', 'Unnamed: 0.1'])

In [6]:
df1.head()

,is_scammer,trx_date,report_date,registereddate,birthday,is_verified,aqc_freq_prepaid_mobile,aqc_mean_prepaid_mobile_amount,aqc_freq_topup,aqc_freq_topup_within_7d,...,gender_Male,gender_None,job_position_KARYAWAN,job_position_LAINNYA,job_position_PEGAWAI_NS,job_position_PELAJAR,job_position_RUMAH_TANGGA,job_position_SPESIALIS,job_position_TIDAK_KERJA,job_position_WIRASWASTA
0,1,0.376934,0.005629,0.673512,0.909517,1,0.019455,0.456958,0.022750,0.007009,...,0,0,0,0,0,1,0,0,0,0
1,0,0.552039,0.632270,0.784524,0.948204,1,0.000000,0.000000,0.038576,0.074766,...,1,0,0,0,0,1,0,0,0,0
2,0,0.772855,0.696998,0.783631,0.766422,1,0.077821,0.324991,0.683482,0.271028,...,1,0,0,0,0,0,0,0,0,1
3,0,0.535865,0.550657,0.565476,0.974102,1,0.042802,0.200904,0.092977,0.042056,...,0,0,0,0,0,1,0,0,0,0
4,0,0.786920,0.715760,0.999702,0.688586,1,0.000000,0.000000,0.031652,0.056075,...,1,0,0,0,0,0,0,0,0,1


In [7]:
# Split Train and Validation Dataset
#from sklearn.model_selection import train_test_split
SPLIT_SIZE = 0.9

train_len = int(len(df1) * SPLIT_SIZE)

shuffled_files = df1.sample(frac = 1, random_state = 1)

test_set = shuffled_files[train_len:]
train_set = shuffled_files[:train_len]

y_train = train_set['is_scammer']
x_train = train_set.drop(columns=['is_scammer'])
y_test = test_set['is_scammer']
x_test = test_set.drop(columns=['is_scammer'])

In [8]:
test_set['weight'] = 1
train_set['weight'] = 1

<ipython-input-8-27bc48f924a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['weight'] = 1
<ipython-input-8-27bc48f924a8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['weight'] = 1


In [9]:
NUMERIC_FEATURE_NAMES = np.array(x_train.columns)
# CATEGORICAL_FEATURE_NAMES = np.array(['is_scammer'])
# Target column name.
TARGET_COLUMN_NAME = "is_scammer"
# The labels of the target columns.
TARGET_LABELS = [1, 0]
# Weight column name.
WEIGHT_COLUMN_NAME = "weight"

In [10]:
def prepare_dataframe(dataframe):
    # Convert the target labels from string to integer.
    dataframe[TARGET_COLUMN_NAME] = dataframe[TARGET_COLUMN_NAME].map(
        TARGET_LABELS.index
    )
    # Cast the categorical features to string.
    # for feature_name in CATEGORICAL_FEATURE_NAMES:
    #     dataframe[feature_name] = dataframe[feature_name].astype(str)


prepare_dataframe(train_set)
prepare_dataframe(test_set)

<ipython-input-10-900af1cb3d39>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[TARGET_COLUMN_NAME] = dataframe[TARGET_COLUMN_NAME].map(


# Gradient Boosted Tree

In [ ]:
# GBT HYPERPARAMETERS HERE
# Maximum number of decision trees. The effective number of trained trees can be smaller if early stopping is enabled.
NUM_TREES = 50
# Minimum number of examples in a node.
MIN_EXAMPLES = 10
# Maximum depth of the tree. max_depth=1 means that all trees will be roots.
MAX_DEPTH = 8
# Ratio of the dataset (sampling without replacement) used to train individual trees for the random sampling method.
SUBSAMPLE = 0.5
# Control the sampling of the datasets used to train individual trees.
SAMPLING_METHOD = "RANDOM"
# Ratio of the training dataset used to monitor the training. Require to be >0 if early stopping is enabled.
VALIDATION_RATIO = 0.1

In [ ]:
def run_experiment(model, train_data, test_data, num_epochs=1, batch_size=None):

    train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(
        train_data, label=TARGET_COLUMN_NAME, weight=WEIGHT_COLUMN_NAME
    )
    test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(
        test_data, label=TARGET_COLUMN_NAME, weight=WEIGHT_COLUMN_NAME
    )

    hist = model.fit(train_dataset, epochs=num_epochs, batch_size=batch_size)
    _, accuracy, precision, recall = model.evaluate(test_dataset, verbose=0)
    print(hist)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test precision: {round(precision * 100, 2)}%")
    print(f"Test recall: {round(recall * 100, 2)}%")

In [ ]:
def specify_feature_usages():
    feature_usages = []

    for feature_name in NUMERIC_FEATURE_NAMES:
        feature_usage = tfdf.keras.FeatureUsage(
            name=feature_name, semantic=tfdf.keras.FeatureSemantic.NUMERICAL
        )
        feature_usages.append(feature_usage)

    # for feature_name in CATEGORICAL_FEATURE_NAMES:
    #     feature_usage = tfdf.keras.FeatureUsage(
    #         name=feature_name, semantic=tfdf.keras.FeatureSemantic.CATEGORICAL
    #     )
    #     feature_usages.append(feature_usage)

    return feature_usages

In [ ]:
def create_gbt_model():
    # See all the model parameters in https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/GradientBoostedTreesModel
    gbt_model = tfdf.keras.GradientBoostedTreesModel(
        features=specify_feature_usages(),
        exclude_non_specified_features=True,
        num_trees=NUM_TREES,
        max_depth=MAX_DEPTH,
        min_examples=MIN_EXAMPLES,
        subsample=SUBSAMPLE,
        validation_ratio=VALIDATION_RATIO,
        task=tfdf.keras.Task.CLASSIFICATION,
    )

    gbt_model.compile(metrics=[
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ])
    return gbt_model

In [ ]:
gbt_model = create_gbt_model()

Use /tmp/tmpb_0kapz4 as temporary training directory


In [ ]:
run_experiment(gbt_model, train_set, test_set)

Reading training dataset...


Training dataset read in 0:00:00.732591. Found 1927 examples.
Training model...
Model trained in 0:00:01.581713
Compiling model...
Model compiled.


Test accuracy: 86.51%
Test precision: 82.76%
Test recall: 91.43%


In [ ]:
gbt_model.summary()

Model: "gradient_boosted_trees_model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (51):
	aqc_freq_prepaid_mobile
	aqc_freq_topup
	aqc_freq_topup_within_7d
	aqc_freq_x2x
	aqc_freq_x2x_within_60d
	aqc_freq_x2x_within_90d
	aqc_mean_prepaid_mobile_amount
	aqc_mean_topup_amount
	aqc_mean_topup_amount_30d
	aqc_mean_topup_amount_7d
	aqc_mean_topup_amount_90d
	aqc_mean_x2x_amount
	aqc_mean_x2x_amount_30d
	aqc_mean_x2x_amount_60d
	aqc_mean_x2x_amount_7d
	aqc_mean_x2x_amount_90d
	aqc_total_topup_amount_7d
	aqc_total_topup_amount_90d
	aqc_total_x2x_amount
	aqc_total_x2x_amount_30d
	aqc_total_x2x_amount_60d
	aqc_total_x2x_amount_7d
	aqc_total_x2x_amount_90d
	avg_other_weight_1
	avg_topup_weight_1
	avg_

# Gradient Boosted Tree #2

In [11]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
def specify_feature_usages():
    feature_usages = []

    for feature_name in NUMERIC_FEATURE_NAMES:
        feature_usage = tfdf.keras.FeatureUsage(
            name=feature_name, semantic=tfdf.keras.FeatureSemantic.NUMERICAL
        )
        feature_usages.append(feature_usage)

    # for feature_name in CATEGORICAL_FEATURE_NAMES:
    #     feature_usage = tfdf.keras.FeatureUsage(
    #         name=feature_name, semantic=tfdf.keras.FeatureSemantic.CATEGORICAL
    #     )
    #     feature_usages.append(feature_usage)

    return feature_usages

In [13]:
# GBT HYPERPARAMETERS HERE
# Maximum number of decision trees. The effective number of trained trees can be smaller if early stopping is enabled.
NUM_TREES = 50
# Minimum number of examples in a node.
MIN_EXAMPLES = 10
# Maximum depth of the tree. max_depth=1 means that all trees will be roots.
MAX_DEPTH = 8
# Ratio of the dataset (sampling without replacement) used to train individual trees for the random sampling method.
SUBSAMPLE = 0.5
# Control the sampling of the datasets used to train individual trees.
SAMPLING_METHOD = "RANDOM"
# Ratio of the training dataset used to monitor the training. Require to be >0 if early stopping is enabled.
VALIDATION_RATIO = 0.1

In [20]:
gbt_model = tfdf.keras.GradientBoostedTreesModel(
        features=specify_feature_usages(),
        exclude_non_specified_features=True,
        num_trees=NUM_TREES,
        max_depth=MAX_DEPTH,
        min_examples=MIN_EXAMPLES,
        subsample=SUBSAMPLE,
        validation_ratio=VALIDATION_RATIO,
        task=tfdf.keras.Task.CLASSIFICATION,
)

gbt_model.compile(metrics=[tf.keras.metrics.BinaryAccuracy(name="accuracy")])

Use /tmp/tmpyuezcslw as temporary training directory


In [21]:
train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(
    train_set, label=TARGET_COLUMN_NAME, weight=WEIGHT_COLUMN_NAME
)
test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(
    test_set, label=TARGET_COLUMN_NAME, weight=WEIGHT_COLUMN_NAME
)

hist = gbt_model.fit(train_dataset, epochs=1, batch_size=None)

Reading training dataset...
Training dataset read in 0:00:01.703556. Found 36645 examples.
Training model...
Model trained in 0:00:16.952626
Compiling model...
Model compiled.


In [22]:
hist.history

{'accuracy': [0.8780219554901123], 'loss': [0.5821880102157593]}

In [23]:
loss, accuracy, f1_score, precision, recall = gbt_model.evaluate(test_dataset, verbose=0)

ValueError: ignored

In [ ]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1_score)

In [24]:
gbt_model.summary()

Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (51):
	aqc_freq_prepaid_mobile
	aqc_freq_topup
	aqc_freq_topup_within_7d
	aqc_freq_x2x
	aqc_freq_x2x_within_60d
	aqc_freq_x2x_within_90d
	aqc_mean_prepaid_mobile_amount
	aqc_mean_topup_amount
	aqc_mean_topup_amount_30d
	aqc_mean_topup_amount_7d
	aqc_mean_topup_amount_90d
	aqc_mean_x2x_amount
	aqc_mean_x2x_amount_30d
	aqc_mean_x2x_amount_60d
	aqc_mean_x2x_amount_7d
	aqc_mean_x2x_amount_90d
	aqc_total_topup_amount_7d
	aqc_total_topup_amount_90d
	aqc_total_x2x_amount
	aqc_total_x2x_amount_30d
	aqc_total_x2x_amount_60d
	aqc_total_x2x_amount_7d
	aqc_total_x2x_amount_90d
	avg_other_weight_1
	avg_topup_weight_1
	avg_

# Gradient Boosted Forest

In [ ]:
num_trees = 25
depth = 5
used_features_rate = 0.5

In [ ]:
def create_forest_model():
    inputs = create_model_inputs()
    features = encode_inputs(inputs)
    features = layers.BatchNormalization()(features)
    num_features = features.shape[1]

    forest_model = NeuralDecisionForest(
        num_trees, depth, num_features, used_features_rate, num_classes
    )

    outputs = forest_model(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
forest_model = create_forest_model()

NameError: ignored

In [ ]:
run_experiment(forest_model)

# Testing

In [27]:
df_test = pd.read_csv("/content/drive/MyDrive/Capstone/datasets/user_data_test_cleaned.csv", sep=';')

In [28]:
df_test = df_test.drop(columns=['Unnamed: 0', 'uid'])

In [29]:
df_test.head()

,trx_date,report_date,registereddate,birthday,is_verified,aqc_freq_prepaid_mobile,aqc_mean_prepaid_mobile_amount,aqc_freq_topup,aqc_freq_topup_within_7d,aqc_mean_topup_amount,...,gender_None,job_position_KARYAWAN,job_position_LAINNYA,job_position_PEGAWAI_NS,job_position_PELAJAR,job_position_RUMAH_TANGGA,job_position_SPESIALIS,job_position_TIDAK_KERJA,job_position_WIRASWASTA,is_scammer
0,0.812236,0.812763,0.963095,0.916693,1,0.000000,0.000000,0.077151,0.053738,0.002067,...,0,0,0,0,1,0,0,0,0,0
1,0.800985,0.801543,1.000000,0.345767,1,0.000000,0.000000,0.015826,0.037383,0.000215,...,0,0,0,0,0,1,0,0,0,1
2,0.796765,0.797335,0.944643,0.962166,1,0.000000,0.000000,0.005935,0.004673,0.000670,...,0,0,0,0,1,0,0,0,0,0
3,0.756681,0.771388,0.872619,0.789122,1,0.038911,0.041929,0.033630,0.065421,0.002565,...,0,1,0,0,0,0,0,0,0,0
4,0.775668,0.776297,0.935714,0.886426,1,0.062257,0.195669,0.016815,0.018692,0.000502,...,0,0,0,0,0,0,0,1,0,1


In [30]:
df_test['weight'] = 1
testing_set = tfdf.keras.pd_dataframe_to_tf_dataset(
    df_test, label=TARGET_COLUMN_NAME, weight=WEIGHT_COLUMN_NAME
)
pred_res = gbt_model.predict(testing_set)

3/3 [==============================] - 0s 39ms/step


In [33]:
pred_res

array([[0.89750856],
       [0.01593702],
       [0.9630184 ],
       ...,
       [0.02497473],
       [0.56845146],
       [0.9200964 ]], dtype=float32)

In [ ]:
TP = 0
FP = 0
TN = 0
FN = 0
for i in range (0,len(pred_res)):
  if pred_res[i] > 0.5:
    if df_test['is_scammer'][i] == 1:
      TP += 1
    else:
      FP += 1
  else:
    if df_test['is_scammer'][i] == 1:
      FN += 1
    else:
      TN += 1

In [ ]:
print("TP: ", TP)
print("FP: ", FP)
print("TN: ", TN)
print("FN: ", FN)

TP:  943
FP:  140
TN:  913
FN:  146


In [ ]:
df_test['is_scammer'][0]

0

In [25]:
gbt_model.save("/content/drive/MyDrive/Capstone/gbt_model")

In [26]:
gbt_model2 = tf.keras.models.load_model("/content/drive/MyDrive/Capstone/gbt_model/")

In [31]:
gbt_model2.predict(testing_set)

3/3 [==============================] - 1s 66ms/step


array([[0.89750856],
       [0.01593702],
       [0.9630184 ],
       ...,
       [0.02497473],
       [0.56845146],
       [0.9200964 ]], dtype=float32)

In [32]:
gbt_model.predict(testing_set)

3/3 [==============================] - 0s 19ms/step


array([[0.89750856],
       [0.01593702],
       [0.9630184 ],
       ...,
       [0.02497473],
       [0.56845146],
       [0.9200964 ]], dtype=float32)

# Steps to Consume GBT Model:

1. Load test dataset.

In [39]:
for i in range (0, len(df_test.columns)-2):
  print(df_test.columns[i])

trx_date
report_date
registereddate
birthday
is_verified
aqc_freq_prepaid_mobile
aqc_mean_prepaid_mobile_amount
aqc_freq_topup
aqc_freq_topup_within_7d
aqc_mean_topup_amount
aqc_mean_topup_amount_7d
aqc_mean_topup_amount_30d
aqc_mean_topup_amount_90d
aqc_total_topup_amount_7d
aqc_total_topup_amount_90d
aqc_freq_x2x
aqc_freq_x2x_within_60d
aqc_freq_x2x_within_90d
aqc_mean_x2x_amount
aqc_mean_x2x_amount_7d
aqc_mean_x2x_amount_30d
aqc_mean_x2x_amount_60d
aqc_mean_x2x_amount_90d
aqc_total_x2x_amount
aqc_total_x2x_amount_7d
aqc_total_x2x_amount_30d
aqc_total_x2x_amount_60d
aqc_total_x2x_amount_90d
dormancy_max_gmt_pay_diff_days
dormancy_mean_gmt_pay_diff_days
dormancy_count_trx
kyc_total_failed
kyc_total_revoked
avg_topup_weight_1
avg_x2x_weight_1
avg_other_weight_1
centrality_outdegree_p2p
centrality_indegree_p2p
centrality_undirected_p2p
centrality_outdegree_sendmoney
gender_Female
gender_Male
gender_None
job_position_KARYAWAN
job_position_LAINNYA
job_position_PEGAWAI_NS
job_position_PELA

2. Setup 'weight' and 'is_scammer' features at default value 0
(gak tau kenapa tapi harus gini)

In [ ]:
df_test['is_scammer'] = 0
df_test['weight'] = 0

3. Set initial parameters.

In [ ]:
# Target column name.
TARGET_COLUMN_NAME = "is_scammer"
# The labels of the target columns.
TARGET_LABELS = [1, 0]
# Weight column name.
WEIGHT_COLUMN_NAME = "weight"

4. Pandas dataframe to tensorflow dataset

In [ ]:
testing_set = tfdf.keras.pd_dataframe_to_tf_dataset(
    df_test, label=TARGET_COLUMN_NAME, weight=WEIGHT_COLUMN_NAME
)

5. Load Model (model GBT ada di drive Capstone -> gbt_model)

In [ ]:
gbt_loaded_model = tf.keras.models.load_model("/path/to/saved/gbt_model")

6. Predict, hasilnya predictionScore. Threshold = 0.5

In [ ]:
gbt_loaded_model.predict(testing_set)